Author: Thrupthi Ann John https://github.com/ThrupthiAnn


# Demo: Calculate the explanation maps used in our paper
Use this notebook to reproduce the quantitative results given in our paper. This is the second demo notebook. First run <b>demo1_maps.ipynb</b>. After running this notebook, run <b>demo3_metrics.ipynb</b>
    
## Step 1: Obtain heatmaps using other methods for comparison
Obtain all the heatmaps you need to compare with canonical saliency maps. We have provided GradCAM, GradCAM++ and ScoreCAM heatmaps for our test images for the recognition model. 
<p> Here are the code links for calculating heatmaps of <a href="https://github.com/utkuozbulak/pytorch-cnn-visualizations/tree/master/src">GradCAM</a>, <a href="https://github.com/adityac94/Grad_CAM_plus_plus">GradCAM++</a> and <a href="https://github.com/utkuozbulak/pytorch-cnn-visualizations/tree/master/src">ScoreCAM</a>
    
<p>CMS maps for models trained on CelebA are provided. 



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread
from skimage.transform import resize
from os.path import join
from os import makedirs
from scipy.io import loadmat
from pathlib import Path

samplefolder = '../data/SampleData'
resultsfolder = '../results'
imagefolder = join(samplefolder, 'SampleImages')
meshfolder = join(samplefolder, 'Sample3D')
cisfolder = join(resultsfolder, 'CIS')
modelfolder= '../data/Models'
frontal = imread(join(modelfolder, 'frontal.jpg'))
frontal_mesh = loadmat(join(modelfolder, 'frontal_mesh.mat'))['vertices'][::20,:]

gradcamfolder = join(samplefolder, 'GradCAM')
gradcamplusfolder = join(samplefolder, 'GradCAMPlus')
scorecamfolder = join(samplefolder, 'ScoreCAM')
cms = np.load(join(modelfolder, 'Recognition_CMS.npy'))

interfolder = join(resultsfolder, 'IntermediateResults')
transcmsfolder = join(interfolder, 'Translated_CMS')
explcmsfolder = join(interfolder, 'Explanation_CMS')
explgradcamfolder = join(interfolder, 'Explanation_GradCAM')
explgradcamplusfolder = join(interfolder, 'Explanation_GradCAMPlus')
explscorecamfolder = join(interfolder, 'Explanation_ScoreCAM')

makedirs(transcmsfolder, exist_ok=True)
makedirs(explcmsfolder, exist_ok = True)
makedirs(explgradcamfolder, exist_ok = True)
makedirs(explgradcamplusfolder, exist_ok = True)
makedirs(explscorecamfolder, exist_ok = True)

p = Path(imagefolder)
filenames = [i.stem for i in p.glob('**/*.jpg')]

## Step 2: Translate CMS maps
Our metrics are calculated after translating the CMS map onto the input face. In this section, we create these translated CMS maps. 
    


In [ ]:
from TranslateCMS import TranslateHeatmap, get_image_numpy

for ii in filenames:
	pimg, sz = get_image_numpy(join(imagefolder, ii+'.jpg'))
	mesh = loadmat(join(meshfolder, ii+'_mesh.mat'))['vertices']
	if len(mesh)>2194:
		mesh = mesh[::20,:]
	newheatmap = TranslateHeatmap(cms, frontal_mesh, mesh, pimg, sz)
	np.save(join(transcmsfolder, ii+'.npy'), newheatmap)

## Step 3: Calculate the Explanation maps
Here, we calculate the reverse explanation maps for all the heatmaps. Before creating the explanation maps the heatmaps are first normalized to a range of [0,1] and the heatmaps for all the methods are standardized to have the same sum of pixels for each image.


In [ ]:
from ExplanationMap import SumNormalize, GetExplanationImage

for ii in filenames:
	img = resize(imread(join(imagefolder, ii+'.jpg')),(224,224))
	cis = resize(np.load(join(transcmsfolder, ii+'.npy')),(224,224))
	gradcam  = resize(np.load(join(gradcamfolder, ii+'.npy')),(224,224))
	gradcamplus = resize(np.load(join(gradcamplusfolder, ii + '.npy')),(224,224))
	scorecam = resize(np.load(join(scorecamfolder, ii+'.npy')),(224,224))
	gradcam = SumNormalize(gradcam, cis)
	gradcamplus = SumNormalize(gradcamplus, cis)
	scorecam = SumNormalize(scorecam, cis)
	cis = SumNormalize(cis, cis)
	GetExplanationImage(img, 1-cis, join(explcmsfolder, ii+'.jpg'))
	GetExplanationImage(img, 1-gradcam, join(explgradcamfolder, ii+'.jpg'))
	GetExplanationImage(img, 1-gradcamplus, join(explgradcamplusfolder, ii+'.jpg'))
	GetExplanationImage(img, 1-scorecam, join(explscorecamfolder, ii+'.jpg'))


View some explanation images

In [ ]:
img = imread(join(explcmsfolder, '000686.jpg'))
plt.imshow(img)